In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, LSTM, Dense

from keras import backend as K
from tensorflow import keras

import numpy as np

In [10]:
import os
from data import generate_train_data

cases = [x for x in os.listdir("dataset/chb-mit-scalp-eeg-database-1.0.0/") if "chb" in x]
# cases = ["chb01", "chb02"]
# generate_train_data(sequence_length=256, stride=256, cases=cases)

In [11]:
from seizure_sequence import SeizureSequence2

seizure_sequence = SeizureSequence2(64, "ml_processed", ["chb01", "chb02", "chb02", "chb03", "chb04", "chb05", "chb06", "chb07"], bias_positive=True)
# data, labels = seizure_sequence.__getitem__(3)
# print(data.shape, labels.shape)

In [12]:
def create_model():
    model = Sequential()

    model.add(Input((23, 256, )))
    model.add(Conv1D(512, kernel_size=3, activation="relu"))
    model.add(MaxPool1D())
    model.add(Conv1D(256, kernel_size=3, activation="relu"))
    model.add(MaxPool1D())
    model.add(Conv1D(128, kernel_size=3, activation="relu"))
    model.add(MaxPool1D())
    # model.add(LSTM(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    return model

model = create_model()

In [13]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 21, 512)           393728    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 10, 512)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 8, 256)            393472    
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 4, 256)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 2, 128)            98432     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 1, 128)          

In [16]:
tf_dataset = seizure_sequence.create_tf_dataset()
model.fit(seizure_sequence, epochs=10)

AttributeError: __enter__

In [ ]:
data = np.load("ml_processed/chb01_03.edf_data.npy")
labels = np.load("ml_processed/chb01_03.edf_labels.npy")
# for i in range(0, len(labels)):
#     if labels[i][0] == 1:
#         print(i)
# print(data.shape)
known_seizures = data[3000:3020]
# print(known_seizures.shape)
known_seizure_label = labels[3000:3020]

for known_seizure in known_seizures:
    # print(known_seizure.shape)
    np.reshape(known_seizure, (-1, 23, 256))
    print(model.predict(known_seizure))

ValueError: in user code:

    File "c:\Users\angus\Documents\cs408-eeg-python\venv3.7\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\angus\Documents\cs408-eeg-python\venv3.7\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\angus\Documents\cs408-eeg-python\venv3.7\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\angus\Documents\cs408-eeg-python\venv3.7\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\angus\Documents\cs408-eeg-python\venv3.7\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\angus\Documents\cs408-ee